In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt


In [ ]:
img1 = cv2.imread('w1.jpg',0)     
img2 = cv2.imread('w2.jpg',0) 

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(20,18))
ax[0].imshow(img1)
ax[1].imshow(img2)

In [ ]:

# Initiate SIFT detector
sift = cv2.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# BFMatcher with default params
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)

# Apply ratio test
good_matches = []
for m,n in matches:
    if m.distance < 0.9*n.distance:
        good_matches.append([m])

# cv2.drawMatchesKnn expects list of lists as matches.
img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good_matches,outImg=None,flags=2)
plt.figure(figsize=(20,18))
plt.imshow(img3)

In [ ]:
pt1=[kp1[m[0].queryIdx].pt for m in good_matches ]
pt2=[kp2[m[0].trainIdx].pt for m in good_matches ]
pt1=np.array(pt1)
pt2=np.array(pt2)

In [ ]:
h,mask=cv2.findHomography(srcPoints=pt1,dstPoints=pt2,method=cv2.FM_RANSAC, ransacReprojThreshold=10)
# h,mask=cv2.findHomography(srcPoints=pt1,dstPoints=pt2,method=cv2.RANSAC, ransacReprojThreshold=5)
print ('Number of detected keypoints after remving outliers: ')
int(sum(mask==1))

In [ ]:
def translatePoint(p, d):
    x, y = p
    dx, dy = d
    return [x + dx, y + dy]


In [ ]:
def homographyTransform(h,p):
    res=h@p
    res=res/res[-1]
    return res[:2].astype('int')

In [ ]:


def drawMatches(pt1, img1, pt2, img2, mask, h_mat):
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    img = np.zeros(shape=(max(h1, h2), w1 + w2), dtype='uint8')
    img[:h1, :w1] = img1
    img[:h2, w1:] = img2
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    x = []
    x.append([0, 0, 1])
    x.append([w1, 0, 1])
    x.append([w1, h1, 1])
    x.append([0, h1, 1])

    tx = [translatePoint(homographyTransform(h_mat, np.array(p)), (w1, 0)) for p in x]
    tx = np.array([tx])
    for i, m in enumerate(mask):
        if m == 1:
            dst = (int(w1 + pt2[i][0]), int(pt2[i][1]))
            src = (int(pt1[i][0]), int(pt1[i][1]))
            #             color=tuple(np.random.randint(0,255,3,dtype='int'))
            color = np.random.randint(0, 255, size=(3), dtype=np.uint8)
            color = (int(color[0]), int(color[1]), int(color[2]))
            cv2.line(img, src, dst, color, thickness=1)
    cv2.polylines(img, tx, isClosed=True, color=color, thickness=2)
    plt.figure(figsize=(20, 18))
    plt.imshow(img)
    plt.show()
    return img


In [ ]:
h

In [ ]:
imgg=drawMatches(pt1,img1,pt2,img2,mask,h)

In [ ]:
def homographyAddTranslation(h,translation):
    h_t=np.eye(3)
    h_t[0,2]=translation[1]
    h_t[1,2]=translation[0]
    return h_t@h

In [ ]:
def stichImages(img1,img2,h):
    dst_h,dst_w=img2.shape[:2]
    dst_shape=(dst_w*3,dst_h*3)
    h=homographyAddTranslation(h,(dst_h,dst_w))
    img=cv2.warpPerspective(img1,h,dst_shape)
    fig,ax=plt.subplots(1,2,figsize=(20,18))
    padded_img=np.pad(img2,[[dst_h,dst_h],[dst_w,dst_w]])
    ax[0].imshow(img)
    ax[1].imshow(padded_img)
    plt.show()
    img_registeded=padded_img.copy()
    img_registeded[img!=0]=img[img!=0]
    plt.figure(figsize=(20,18))
    plt.imshow(img_registeded)
    plt.show()
    return img_registeded
    

In [ ]:
result=stichImages(img1,img2,h)